In [11]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
from time import time
from scipy import stats
import matplotlib.pyplot as plt


mag_v_sun = -26.72              #Sun's V magnitude of Gaia   Jordi et al. 2010

## Definition functions

### Model Magnitud

In [ ]:
def model_magnitud(theta, data):
    mag_v_sun = -26.72

    phase_angle = data['solar_phase_angle']
    range_sat = data['range_sat']
    cross_section = theta['cross_section']
    albedo = theta['albedo'] 
    mix_coef = theta['mix_coef']

   # Change degrees to radian
    degtorad = lambda x : x*np.pi/180.

   # Definition coefficients of equation
    coe_1 = 2/(3*np.pi)
    coe_2 = 1/(4*np.pi)

    phase_angle_rad = degtorad(phase_angle)	

   
    part1 = (coe_1 *((np.pi - phase_angle_rad)*np.cos(phase_angle_rad) - np.sin(phase_angle_rad))) - coe_2
   
    model_value = mag_v_sun - 2.5 * np.log10(cross_section * albedo * (mix_coef * part1) + coe_2) + 5*np.log10(range_sat)
    
    frame = {'model_value': model_value}
    model_value_est = pd.DataFrame(frame)


    return model_value_est

### Sum Square

In [ ]:
def sum_squares(theta, data):

    model_value = model_magnitud(theta, data).model_value
    rest = (data['mag_observation'] + data['e_mag']) + model_value
    residual = (rest ** 2).sum(axis=0)/(rest.shape[0] - theta.shape[1])
    estimator = [residual, rest]
    
    return estimator

### Markov Chains

In [ ]:
def markov_chains(init_point, std_point, size_chain):
    chain = np.random.normal(init_point, std_point, size_chain).round(3)
    
    return chain

### Beta distribution

In [ ]:
def beta(x):
    sample_beta = stats.beta(1,1)
    r = sample_beta.rvs(x)
    return r

### Read Data

#### PC

In [ ]:
path = '/home/kero/Documents/PhD/Re-analyse/result_analyse.csv'

data_sat = pd.read_csv(path)

data_in = pd.DataFrame()
data_in['solar_phase_angle'] = data_sat['obs_phase_angle']
data_in['range_sat'] = data_sat['range_sat']
data_in['mag_observation'] = data_sat['mag_observation']
data_in['e_mag'] = data_sat['mag_observation_e']
n = data_in.shape[0]

#### Laptop

In [ ]:
path = '/home/kero/Documents/PhD/re-analyse/test.csv'

data_sat = pd.read_csv(path)
data_in = pd.DataFrame()
data_in['solar_phase_angle'] = data_sat['SunPhaseAng']
data_in['range_sat'] = data_sat['ObsTrailLength']
data_in['mag_observation'] = data_sat['ObsMag']
data_in['e_mag'] = data_sat['e_ObsMag']

### Markov chain Monte Carlo

In [ ]:
p_value = 0.83
num_simulations = 2000

good_p_value = []
mixing_coeffient_est = []
albedo_est = []
cross_section_est = []

mix_init_point = 0.75
mix_std_point = 0.12
cross_init_point = 0.8
cross_std_point = 0.35
albedo_init_point = 0.55
albedo_std_point = 0.2

size_chain_in = data_in.shape[0]

for i in range(num_simulations):
    mix_coef_markov = markov_chains(mix_init_point, mix_std_point, size_chain_in)
    albedo_markov = markov_chains(albedo_init_point, albedo_std_point, size_chain_in)
    cross_section_markov = markov_chains(cross_init_point, cross_std_point, size_chain_in)

    mix_coef = np.random.choice(mix_coef_markov)
    albedo = np.random.choice(albedo_markov)
    cross_section = np.random.choice(cross_section_markov)

    

    parameters = pd.DataFrame(data={'mix_coef': mix_coef,
                            'albedo': albedo,
                            'cross_section' : cross_section},
                            index=range(data_in.shape[0]))

    model_value = model_magnitud(parameters, data_in)

    estimador = sum_squares(parameters, data_in)



